In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/MyDrive/VLSP2021-MRC-BLANC'

/content/gdrive/.shortcut-targets-by-id/1WQcuxpxeN4XEvs7OGyXFqtypm6n8iNvd/VLSP2021-MRC-BLANC


In [ ]:
!pip install datasets sacremoses huggingface_hub tokenizers sentencepiece

     |████████████████████████████████| 290 kB 5.1 MB/s 
     |████████████████████████████████| 895 kB 45.9 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 3.3 MB 43.8 MB/s 
     |████████████████████████████████| 1.2 MB 45.8 MB/s 
     |████████████████████████████████| 125 kB 57.3 MB/s 
     |████████████████████████████████| 243 kB 73.7 MB/s 
     |████████████████████████████████| 1.3 MB 52.8 MB/s 
     |████████████████████████████████| 271 kB 76.5 MB/s 
     |████████████████████████████████| 160 kB 71.5 MB/s 


# Predict

In [ ]:
!python run_squad.py \
  --model_type xlm-roberta \
  --model_name_or_path ./models/finetuned10_models/xlm-roberta-large-fold_1 \
  --do_eval \
  --predict_file ./data/private_test_data/private_test_syllable.json \
  --per_gpu_eval_batch_size 4 \
  --max_seq_length 384 \
  --max_query_length 128 \
  --max_answer_length 500 \
  --doc_stride 128 \
  --geometric_p 0.7 \
  --window_size 2 \
  --lmb 0.4 \
  --version_2_with_negative \
  --overwrite_cache \
  --cache_dir ./results/private_test_result/10_fold/xlm-roberta-large-fold_1 \
  --output_dir ./results/private_test_result/10_fold/xlm-roberta-large-fold_1 \

In [ ]:
!python run_squad.py \
  --model_type rembert \
  --model_name_or_path ./models/finetuned10_models/rembert-fold_1 \
  --do_eval \
  --predict_file ./data/private_test_data/private_test_syllable.json \
  --per_gpu_eval_batch_size 2 \
  --max_seq_length 384 \
  --max_query_length 128 \
  --max_answer_length 500 \
  --doc_stride 128 \
  --geometric_p 0.7 \
  --window_size 2 \
  --lmb 0.4 \
  --logging_steps 500 \
  --save_steps 0 \
  --version_2_with_negative \
  --overwrite_cache \
  --cache_dir ./results/private_test_result/10_fold/rembert-fold_1 \
  --output_dir ./results/private_test_result/10_fold/rembert-fold_1\

# Ensemble

In [ ]:
n_fold = 10
input_nbest_files_xlm = [f"./results/private_test_result/10_fold/xlm-roberta-large-fold_{i+1}/nbest_predictions_.json" for i in range(n_fold)]
input_nbest_files_rembert = [f"./results/private_test_result/10_fold/rembert-fold_{i+1}/nbest_predictions_.json" for i in range(n_fold)]
input_nbest_files=input_nbest_files_rembert + input_nbest_files_xlm 

In [ ]:
import collections
import json

idx = 0
best_cof = [0.5/10 for _ in range(10)] + [0.5/10 for _ in range(10)]
 
all_nbest = collections.OrderedDict()
for input_file in input_nbest_files:
    with open(input_file, "r") as reader:
        input_data = json.load(reader, strict=False)
        for (key, entries) in input_data.items():
            if key not in all_nbest:
                all_nbest[key] = collections.defaultdict(float)
            for entry in entries:
                all_nbest[key][entry["text"]] += best_cof[idx] * entry["probability"]
    idx += 1

output_predictions = {}
for (key, entry_map) in all_nbest.items():
    sorted_texts = sorted(
        entry_map.keys(), key=lambda x: entry_map[x], reverse=True)
    best_text = sorted_texts[0]
    output_predictions[key] = best_text

In [ ]:
file_out = "/content/results_pred.json"
with open(file_out, 'w', encoding='utf-8') as outfile:
        json.dump(output_predictions, outfile ,ensure_ascii=False,indent=4)

# Keep the highest probability among duplicate answer

In [ ]:
def group_ans_in_context(test_file):
  public_test = json.load(open(test_file))['data']
  i = 0
  data_id = {}
  for dt in public_test:
    for para in dt['paragraphs']:
      a = []
      for qa in para['qas']:
        a.append(qa['id'])
      data_id[i] = a
      i += 1
  return data_id
 
def find_best_prediction(all_nbest):
  best_predictions = {}
  for (key, entry_map) in all_nbest.items():
      sorted_texts = sorted(
          entry_map.keys(), key=lambda x: entry_map[x], reverse=True)
      best_text = sorted_texts[0]
      best_predictions[key] = entry_map[best_text]
  return best_predictions

def find_same_predict_answer(data_id, data, best_predictions ):
    occurrences = lambda s, lst: (i for i,e in enumerate(lst) if e == s)
    dct_final = {}
    for key, values in data_id.items():
        predict = []
        for value in values:
          predict.append(data[value])
        lst_same_ans = []
        flag = False
        for value in values:
          if data[value] != "":
            lst = list(occurrences(data[value], predict))
            if len(lst) > 1:
              d = {}
              for i in lst:
                flag = True
                d[values[i]] = best_predictions[values[i]]
              if d not in lst_same_ans:
                lst_same_ans.append(d)
        if flag == True:
          dct_final[key] = lst_same_ans
        flag = False
    return dct_final

def find_null_ans(dct_final): 
    """Find the worst predict answer among the same answer in each context"""
    null_ans = []
    for key, values in dct_final.items():
      for value in values:
        sorted_texts = sorted(
            value.keys(), key=lambda x: value[x], reverse=True)
        worst_text = sorted_texts[-1]
        null_ans.append(worst_text)
        if len(sorted_texts)>2:
          worst_text = sorted_texts[-2]
          null_ans.append(worst_text)
    return null_ans

def delete_answers(predict_file_path, test_file_path, all_nbest):
    data = json.load(open(predict_file_path)) # Load data in predicted file
    data_id = group_ans_in_context(test_file_path) # Group answers id in the same context together
    best_prediction = find_best_prediction(all_nbest) # Find the highest probability predict answer in each context
    dct_final = find_same_predict_answer(data_id, data, best_prediction) # Find the probability in the same answer in each context
    null_ans = find_null_ans(dct_final) # Find the worst predict answer among the same answer in each context
    for k, v in data.items(): 
      if k in null_ans:
        data[k] = ""
    return data

In [ ]:
results = delete_answers("/content/results_pred.json", "./data/private_test_data/private_test_syllable.json", all_nbest)

In [ ]:
file_out = "/content/results.json"
with open(file_out, 'w', encoding='utf-8') as outfile:
        json.dump(results, outfile ,ensure_ascii=False,indent=4)